# 6. Optimización de hiper-parámetros

¡Bienvenidos a la sexta sesión! Ahora que ya sabemos lo que son las redes neuronales, las redes convolucionales y todos sus parámetros y entresijos, vamos a ver cómo podemos optimizarlas al máximo.

Para ello, hoy veremos:

**Optimización de hiper-parámetros**
* Grid search
* Hyperopt
* Algoritmos genéticos


## 6.1 Grid search

El grid search es el método más sencillo que existe para encontrar los mejores parámetros dentro de un conjunto. Esencialmente se trata de fuerza bruta. Vamos a ver un ejemplo sencillo:

Suponed que tenemos nuestra anterior red y queremos ver qué parámetros son los mejores. Tenemos:

* dropout: puede variar de 0 a 0.5 en intervalos 0.1
* learning rate: puede variar de 0.1 a 0.001 en intervalos de x10
* número de filtros: puede variar de 64 a 256 en intervalos de 64
* tamaños del filtro: puede variar desde 3 hasta 7 de 2 en 2 (filtros cuadrados siempre)

Pues bien, lo que el grid search haría es los iguiente:

In [1]:
# definimos nuestra grid
dropouts = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
learning_rates = [0.1, 0.01, 0.001, 0.0001]
n_filters = [64, 128, 192, 256]
filter_sizes = [3, 5, 7]

# creamos una variable para guardar nuestras accuracies
log_accuracies = []

# para hacer esta prueba definimos una función que sustituye a nuestra red y 
# asigna un valor aleatorio cada vez
# si lo hiciésemos con nuestra red de verdad, tardaría demasiado como para verlo
# en clase
from random import uniform
def dummy_net(d, lr, nf, fs):
  print('Ejecutando la red con d={}, lr={}, nf={}, fs={}'.format(d, lr, nf, fs))
  return uniform(0,1)

# contador
i = 1

# empezamos el grid search
for d in dropouts:
  for lr in learning_rates:
    for nf in n_filters:
      for fs in filter_sizes:
        result_net = dummy_net(d, lr, nf, fs)
        print('[{}] Resultado: {}'.format(i, result_net))
        log_accuracies.append(result_net)
        i += 1

Ejecutando la red con d=0, lr=0.1, nf=64, fs=3
[1] Resultado: 0.8079891398690431
Ejecutando la red con d=0, lr=0.1, nf=64, fs=5
[2] Resultado: 0.8505819087417151
Ejecutando la red con d=0, lr=0.1, nf=64, fs=7
[3] Resultado: 0.28810773410744495
Ejecutando la red con d=0, lr=0.1, nf=128, fs=3
[4] Resultado: 0.7807722174811716
Ejecutando la red con d=0, lr=0.1, nf=128, fs=5
[5] Resultado: 0.8146287045398535
Ejecutando la red con d=0, lr=0.1, nf=128, fs=7
[6] Resultado: 0.8448586950417052
Ejecutando la red con d=0, lr=0.1, nf=192, fs=3
[7] Resultado: 0.9931101298363152
Ejecutando la red con d=0, lr=0.1, nf=192, fs=5
[8] Resultado: 0.382895321177511
Ejecutando la red con d=0, lr=0.1, nf=192, fs=7
[9] Resultado: 0.9266662116112417
Ejecutando la red con d=0, lr=0.1, nf=256, fs=3
[10] Resultado: 0.09842404394836868
Ejecutando la red con d=0, lr=0.1, nf=256, fs=5
[11] Resultado: 0.9416535471767618
Ejecutando la red con d=0, lr=0.1, nf=256, fs=7
[12] Resultado: 0.8848411814922239
Ejecutando la r

In [2]:
# y el mejor resultado sería
import numpy as np
idx_max = np.nonzero(log_accuracies==np.max(log_accuracies))
print(idx_max)

print('Best execution: {}. Accuracy: {}'.format(idx_max[0][0], log_accuracies[idx_max[0][0]]))

(array([232]),)
Best execution: 232. Accuracy: 0.9977810539100487


Y con esto sabríamos cuál es la mejor configuración de nuestra red. Mucho mejor que andar cambiando parámetros a mano verdad?

Cuál es el problema de este método? Pues que tenemos 6 x 4 x 4 x 3 ejecuciones de nuestra red, lo que hacen un total de 288 ejecuciones, a un mínimo de 10 minutos por ejecución, son 48 horas, osea, 2 días!!!

Si queréis comprobarlo no tenéis más que meter dentro de dummy_net la arquitectura de la red y ejecutarlo ;-)

Vamos a hacer una prueba con una red muy sencillita para que veáis qué tal funciona:

In [0]:
# importamos los paquetes necesarios
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.optimizers import Adam

# cargamos los datos y pasamos de vector a imagen
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# normalizamos
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

def net(do, lr, nf, fs):

  model = Sequential()
  model.add(Conv2D(nf, kernel_size=fs, activation='relu', input_shape=input_shape))
  model.add(Conv2D(nf, fs, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(do))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(do))
  model.add(Dense(10, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])

  model.fit(x_train, y_train,
            batch_size=1024,
            epochs=1,
            verbose=0,
            validation_data=(x_test, y_test))

  score = model.evaluate(x_test, y_test, verbose=0)
  
  print('Red con d={}, lr={}, nf={}, fs={}. Loss: {}. Acc: {}.'.format(d, lr, nf, fs, score[0], score[1]))      
  return score[1] # accuracy

In [17]:
# definimos nuestra grid
dropouts = [0, 0.3]
learning_rates = [0.1, 0.01]
n_filters = [32, 64]
filter_sizes = [3, 5]

# creamos una variable para guardar nuestras accuracies
log_accuracies = []

# contador
i = 1

# empezamos el grid search
for d in dropouts:
  for lr in learning_rates:
    for nf in n_filters:
      for fs in filter_sizes:
        result_net = net(d, lr, nf, fs)
        print('[{}] Resultado: {}'.format(i, result_net))
        log_accuracies.append(result_net)
        i += 1
        
# y el mejor resultado sería
import numpy as np
idx_max = np.nonzero(log_accuracies==np.max(log_accuracies))
print(idx_max)

print('Best execution: {}. Accuracy: {}'.format(idx_max[0][0], log_accuracies[idx_max[0][0]]))

Red con d=0, lr=0.1, nf=32, fs=3. Loss: 14.538521838378907. Acc: 0.098.
[1] Resultado: 0.098
Red con d=0, lr=0.1, nf=32, fs=5. Loss: 14.454707762145997. Acc: 0.1032.
[2] Resultado: 0.1032
Red con d=0, lr=0.1, nf=64, fs=3. Loss: 14.461155032348632. Acc: 0.1028.
[3] Resultado: 0.1028
Red con d=0, lr=0.1, nf=64, fs=5. Loss: 14.454707762145997. Acc: 0.1032.
[4] Resultado: 0.1032
Red con d=0, lr=0.01, nf=32, fs=3. Loss: 0.06582791985776275. Acc: 0.9787.
[5] Resultado: 0.9787
Red con d=0, lr=0.01, nf=32, fs=5. Loss: 0.07418846949008294. Acc: 0.9751.
[6] Resultado: 0.9751
Red con d=0, lr=0.01, nf=64, fs=3. Loss: 0.05427608981369995. Acc: 0.9821.
[7] Resultado: 0.9821
Red con d=0, lr=0.01, nf=64, fs=5. Loss: 0.06231040436052717. Acc: 0.9802.
[8] Resultado: 0.9802
Red con d=0.3, lr=0.1, nf=32, fs=3. Loss: 14.454707762145997. Acc: 0.1032.
[9] Resultado: 0.1032
Red con d=0.3, lr=0.1, nf=32, fs=5. Loss: 12.918653588867187. Acc: 0.1985.
[10] Resultado: 0.1985
Red con d=0.3, lr=0.1, nf=64, fs=3. Los

Bueno, no está mal, verdad?

Aunque estaría genial que hubiesen métodos más rápidos o usando algo de heurística, no?? En vez de fuerza bruta...

Pues estáis de suerte! Existen varios métodos de este tipo:

* Spearmint (Python)
* BayesOpt (C++ with Python and Matlab/Octave interfaces)
* hyperopt (Python)
* SMAC (Java)
* REMBO (Matlab)
* MOE (C++/Python)

+INFO: http://fastml.com/optimizing-hyperparams-with-hyperopt/

Y hoy vamos a ver **hyperopt**!!

## 6.2 Hyper-opt

Otra opción es utilizar hyperopt (Hyperopt: Distributed Asynchronous Hyper-parameter Optimization, https://github.com/hyperopt/hyperopt).

Hyperopt es una librería escrita en Python que permite optimizar funciones de una forma rápida fijándose más en los valores que más probablemente van a dar una buena solución.

Actualmente tiene dos algoritmos implementados para hacer esto:

* Random Search
* Tree of Parzen Estimators (TPE)

Además, se pueden ejecutar en serie o en paralelo, haciendo uso de MongoDB.

Vamos a ver un ejemplo de cómo utilizarlo:

In [18]:
# instalamos los paquetes necesarios
!pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
!pip install hyperopt

# imports necesarios
import sys
import time
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import max_norm
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10

SEED = 42

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
validation_split = 0.1
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split, random_state=SEED)

# como os dije, en la práctica no se normaliza con media y std, simplemente se
# divide entre 255. Esto se debe a las características de una imagen, que sabemos
# que solo puede tener valores entre 0 y 255
# Primero convertimos a float y luego dividimos (si no convirtiesemos a float
# perderiamos toda la info de decimales!!! )
X_train = X_train.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# convertimos las etiquetas a one-hot encoding
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_val = to_categorical(y_val, n_classes)
y_test = to_categorical(y_test, n_classes)

# definimos nuestro espacio de búsqueda
# vamos a variar:
# - el número de filtros en nuestras capas conv
# - el porcentaje de dropout
# - el número de neuronas en la capa dense
space = {
    'n_filters_conv': hp.choice('n_filters_conv', [32, 64, 128]),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [256, 512, 1024]), 
}

def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)
  model = Sequential()
  
  # Primer bloque convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Segundo bloque convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Tercer bloque convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Bloque clasificador
  model.add(Flatten())
  model.add(Dense(pars['neurons_dense'], activation='relu', kernel_constraint=max_norm(3.)))
  model.add(Dropout(pars['dropout']))
  model.add(Dense(10, activation='softmax'))

  # Compilamos el modelo
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=0.0001, decay=1e-6),
                metrics=['accuracy'])

  # Entrenamos el modelo
  history = model.fit(X_train, 
                      y_train,
                      batch_size=128,
                      shuffle=True,
                      epochs=5,
                      validation_data=(X_val, y_val),
                      verbose = 1)

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])
  best_val_acc = np.max(history.history['val_acc'])
  
  print('Epoch {} - val acc: {} - val loss: {}'.format(best_epoch_loss, best_val_acc, best_val_loss))
  sys.stdout.flush()
  
  return {'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

    100% |████████████████████████████████| 1.3MB 7.3MB/s 
  Found existing installation: networkx 2.1
    Uninstalling networkx-2.1:
      Successfully uninstalled networkx-2.1
    100% |████████████████████████████████| 102kB 3.2MB/s 
    100% |████████████████████████████████| 163kB 5.9MB/s 
    100% |████████████████████████████████| 389kB 7.3MB/s 
  Running setup.py bdist_wheel for hyperopt ... - \ done
  Stored in directory: /content/.cache/pip/wheels/32/69/f5/3267146c22e76dbf8c5e13a535d3c00b9efabe58883a0da65d
Successfully built hyperopt
 60153856/170498071 [=========>....................] - ETA: 2s

170500096/170498071 [==============================] - 7s 0us/step
Parameters:  {'dropout': 0.07803597686324265, 'n_filters_conv': 32, 'neurons_dense': 1024}
Train on 45000 samples, validate on 5000 samples
Epoch 1/5
45000/45000 [==============================] - 11s 234us/step - loss: 2.1185 - acc: 0.2134 - val_loss: 1.9044 - val_acc: 0.2946
Epoch 2/5
21248/45000 [=============>................] - ETA: 3s - loss: 1.8605 - acc: 0.3191

45000/45000 [==============================] - 8s 168us/step - loss: 1.8160 - acc: 0.3371 - val_loss: 1.7890 - val_acc: 0.3368
Epoch 3/5
45000/45000 [==============================] - 8s 169us/step - loss: 1.6981 - acc: 0.3796 - val_loss: 1.6422 - val_acc: 0.3922
Epoch 4/5
45000/45000 [==============================] - 8s 167us/step - loss: 1.6277 - acc: 0.4080 - val_loss: 1.6042 - val_acc: 0.3986
Epoch 5/5
19328/45000 [===========>..................] - ETA: 4s - loss: 1.5807 - acc: 0.4254

45000/45000 [==============================] - 8s 167us/step - loss: 1.5729 - acc: 0.4270 - val_loss: 1.5412 - val_acc: 0.4386
Epoch 4 - val acc: 0.4386 - val loss: 1.5411990602493286
Parameters:  {'dropout': 0.492927011226561, 'n_filters_conv': 32, 'neurons_dense': 256}
Train on 45000 samples, validate on 5000 samples
Epoch 1/5
45000/45000 [==============================] - 10s 226us/step - loss: 2.3195 - acc: 0.1056 - val_loss: 2.2997 - val_acc: 0.1338
Epoch 2/5
29184/45000 [==================>...........] - ETA: 2s - loss: 2.2599 - acc: 0.1247

45000/45000 [==============================] - 8s 176us/step - loss: 2.2335 - acc: 0.1401 - val_loss: 2.1812 - val_acc: 0.1936
Epoch 3/5
45000/45000 [==============================] - 8s 172us/step - loss: 2.0856 - acc: 0.1965 - val_loss: 2.0412 - val_acc: 0.2320
Epoch 4/5
45000/45000 [==============================] - 8s 173us/step - loss: 1.9781 - acc: 0.2331 - val_loss: 1.9596 - val_acc: 0.2640
Epoch 5/5
19456/45000 [===========>..................] - ETA: 4s - loss: 1.9340 - acc: 0.2536

...

45000/45000 [==============================] - 11s 246us/step - loss: 2.1309 - acc: 0.2080 - val_loss: 1.9672 - val_acc: 0.2730
Epoch 2/5
45000/45000 [==============================] - 8s 176us/step - loss: 1.8591 - acc: 0.3170 - val_loss: 1.8075 - val_acc: 0.3262
Epoch 3/5
45000/45000 [==============================] - 8s 174us/step - loss: 1.7544 - acc: 0.3578 - val_loss: 1.7085 - val_acc: 0.3602
Epoch 4/5
19584/45000 [============>.................] - ETA: 4s - loss: 1.6987 - acc: 0.3792

45000/45000 [==============================] - 8s 167us/step - loss: 1.6808 - acc: 0.3844 - val_loss: 1.6398 - val_acc: 0.3986
Epoch 5/5
45000/45000 [==============================] - 8s 169us/step - loss: 1.6288 - acc: 0.4025 - val_loss: 1.5824 - val_acc: 0.4174
Epoch 4 - val acc: 0.4174 - val loss: 1.582425901031494
Parameters:  {'dropout': 0.10589221051899217, 'n_filters_conv': 64, 'neurons_dense': 512}
Train on 45000 samples, validate on 5000 samples
Epoch 1/5
30592/45000 [===================>..........] - ETA: 4s - loss: 2.1223 - acc: 0.2070

45000/45000 [==============================] - 15s 324us/step - loss: 2.0482 - acc: 0.2376 - val_loss: 1.8401 - val_acc: 0.3248
Epoch 2/5
45000/45000 [==============================] - 11s 238us/step - loss: 1.7700 - acc: 0.3489 - val_loss: 1.6925 - val_acc: 0.3764
Epoch 3/5
39552/45000 [=========================>....] - ETA: 1s - loss: 1.6523 - acc: 0.3955

45000/45000 [==============================] - 11s 235us/step - loss: 1.6454 - acc: 0.3975 - val_loss: 1.5591 - val_acc: 0.4326
Epoch 4/5
45000/45000 [==============================] - 11s 236us/step - loss: 1.5451 - acc: 0.4319 - val_loss: 1.4760 - val_acc: 0.4650
Epoch 5/5
42624/45000 [===========================>..] - ETA: 0s - loss: 1.4855 - acc: 0.4578

45000/45000 [==============================] - 10s 223us/step - loss: 1.4848 - acc: 0.4586 - val_loss: 1.4323 - val_acc: 0.4810
Epoch 4 - val acc: 0.481 - val loss: 1.4323065658569336
{'dropout': 0.2637507835573508, 'n_filters_conv': 2, 'neurons_dense': 0}


In [27]:
trials.trials

[{'book_time': datetime.datetime(2018, 6, 16, 18, 56, 15, 517000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'dropout': [0], 'n_filters_conv': [0], 'neurons_dense': [0]},
   'tid': 0,
   'vals': {'dropout': [0.07803597686324265],
    'n_filters_conv': [0],
    'neurons_dense': [2]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2018, 6, 16, 18, 56, 57, 300000),
  'result': {'best_epoch': 4,
   'eval_time': 1529175417.3004055,
   'history': <keras.callbacks.History at 0x7fc9b4b16518>,
   'loss': 1.5411990602493286,
   'model': <keras.models.Sequential at 0x7fc9f41c60f0>,
   'status': 'ok'},
  'spec': None,
  'state': 2,
  'tid': 0,
  'version': 0},
 {'book_time': datetime.datetime(2018, 6, 16, 18, 56, 57, 305000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'dropout': [1], 'n_filters_conv': [1], 'neurons_dense': [1]},
   'tid': 1,
   'vals': {'dropout': [0.49292701

In [34]:
trials.trials[3]

{'book_time': datetime.datetime(2018, 6, 16, 18, 58, 52, 4000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'dropout': [3], 'n_filters_conv': [3], 'neurons_dense': [3]},
  'tid': 3,
  'vals': {'dropout': [0.2637507835573508],
   'n_filters_conv': [2],
   'neurons_dense': [0]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2018, 6, 16, 19, 0, 3, 656000),
 'result': {'best_epoch': 4,
  'eval_time': 1529175603.6567712,
  'history': <keras.callbacks.History at 0x7fc9b33de518>,
  'loss': 1.3960030067443847,
  'model': <keras.models.Sequential at 0x7fc9b3582f60>,
  'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 3,
 'version': 0}

In [28]:
trials.results

[{'best_epoch': 4,
  'eval_time': 1529175417.3004055,
  'history': <keras.callbacks.History at 0x7fc9b4b16518>,
  'loss': 1.5411990602493286,
  'model': <keras.models.Sequential at 0x7fc9f41c60f0>,
  'status': 'ok'},
 {'best_epoch': 4,
  'eval_time': 1529175459.8353844,
  'history': <keras.callbacks.History at 0x7fc9b42fbeb8>,
  'loss': 1.8937722667694092,
  'model': <keras.models.Sequential at 0x7fc9b4bc3400>,
  'status': 'ok'},
 {'best_epoch': 4,
  'eval_time': 1529175531.999088,
  'history': <keras.callbacks.History at 0x7fc9f9c09208>,
  'loss': 1.4245373298645019,
  'model': <keras.models.Sequential at 0x7fc9b43332b0>,
  'status': 'ok'},
 {'best_epoch': 4,
  'eval_time': 1529175603.6567712,
  'history': <keras.callbacks.History at 0x7fc9b33de518>,
  'loss': 1.3960030067443847,
  'model': <keras.models.Sequential at 0x7fc9b3582f60>,
  'status': 'ok'},
 {'best_epoch': 4,
  'eval_time': 1529175675.4188085,
  'history': <keras.callbacks.History at 0x7fc9b2a6d470>,
  'loss': 1.497109185

In [29]:
trials.losses()

[1.5411990602493286,
 1.8937722667694092,
 1.4245373298645019,
 1.3960030067443847,
 1.4971091859817505,
 1.6442922735214234,
 1.4325138746261596,
 1.5265537229537964,
 1.582425901031494,
 1.4323065658569336]

Qué os parece? mola eh? Así, podéis dejar vuestra configuración e iros a hacer algo más útil que andar variando parámetros hasta que encontréis la configuración adecuada.

**Mejor que lo hagan por nosotros y que nos la encontremos automáticamente seleccionada!**

Pero no tenemos por qué quedarnos aquí, podemos también variar el número de capas o si queremos conexiones residuales, por ejemplo! Sí, esto significa que podemos variar la **arquitectura** también!!

Aquí tenéis un ejemplo muy completo: https://github.com/Vooban/Hyperopt-Keras-CNN-CIFAR-100

Y aquí otro que quizá os parezca interesante: 

<img src="http://cdn.lizamayliza.com/storage/cache/images/003/636/Daffy-Duck-Money-eyes-feature,xlarge.1480369578.jpg" border="0" height="200">

https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-hyperparameters-optimization-cb2b4a29b8ee

## 6.3 Algoritmos genéticos

Quizás estéis pensando en que tras haber visto esto no necesitáis saber nada más, ya habéis encontrado algo que es la repera! Pues atentos porque lo que viene, pese a ser algo muy sencillo, es potentísimo, y os va encantar. Sí, estoy hablando de **los algoritmos genéticos**.

En esencia, los algoritmos genéticos son un método de búsqueda meta-heurística inspirados en la evolución natural. Pertenecen a los algoritmos evolutivos, concretamente a los Algoritmos de búsqueda aleatoria guiada (Guided Random Search algorithms (Evolutionary Alg.)).

Esto que os puede parecer chino, es muy sencillo. Vamos a entenderlos con un ejemplo:

<img src="https://image.ibb.co/cJcQYJ/ga_problem.png" alt="ga_problem" border="0">

Imaginad que tenemos un puzzle, y nos queda solo una pieza por encajar. Lo que pasa es que este puzzle es muy especial, porque nos permite fabricarnos nuestras propias piezas. Para ello, disponemos de varios mecanismos:

* **combinar** partes de piezas (**crossover** o recombinación)
* **modificar** determinadas partes de esas piezas (**mutación**)
* **escoger** las mejores piezas de todas las que hemos hecho, para a partir de ellas, construir nuevas que sean mejores (**selección**)

Entonces imaginaos que decidimos cortar 10 trozos de carton, que son nuestras 10 piezas iniciales con las que vamos a probar a ver si alguna encaja perfectamnte. Las probamos todas, y de esas 10, 5 encajan más o menos. Así que seleccionamos esas 5 y fabricamos nuevas a partir de ellas mediante los mecanismos explicados arriba:

* de las 5 seleccionadas, sacamos 5 más combinando partes de dos de esas 5 originales escogiéndolas aleatoriamente
* de las 5 originales, y las nuevas 5 que nos hemos creado, sacamos 5 más modificando ligeramente una de las puntas de la pieza

Ahora resulta que tenemos 15 piezas, y nosotros queremos tener siempre 10, porque si no a la 5 vez que hiciéramos esto tendríamos una barbaridad de piezas!! Así que:

* probamos nuestras 15 piezas y nos quedamos con la que mejor encaja, y luego 9 escogidas al azar

Pues ya está, eso es lo que hace un algoritmo genético!! Ya sabéis cómo funcionan!! Sencillo, verdad? Pues no os hacéis una idea de lo potentes que son :-)

Vamos a verlo un poco más concreto siguiendo con nuestro ejemplo:

<img src="https://image.ibb.co/b7ySfy/geneticos_puzzle.png" alt="geneticos_puzzle" border="0">

Como podéis ver:

* cada pieza de nuestro conjunto de piezas (población) es un **cromosoma**
* cada parte de nuestra pieza es un **gen**, por tanto, nuestro cromosoma tiene 4 genes
* los posibles valores o configuraciones que puede tener cada gen se llama **alelo**

Esto es exactamente igual que en biología! No os dije que estaban inspirados en la evolución natural? :-)

Vale, pues vamos a ir relacionando estas palabrejas con nuestro ejemplo:

* necesitamos encontrar la pieza correcta para nuestro hueco del puzzle
* tenemos un conjunto inicial de piezas (**población**) que pueden encajar bien o no, no lo sabemos
* comprobamos cómo de bien encajan estas piezas (usando la **función de fitness**)
* si ninguna de las piezas encaja como nos gustaría, modificamos las piezas (con los operadores: **crossover** y **mutación**)
* comprobamos como de bien encajan las piezas recien creadas (**función de fitness**)
* seleccionamos las piezas que queremos aguantar para la próxima iteración (**selección**)

* y volvemos a empezar! así, hasta que enocntremos una pieza que encaje con la precisión que nosotros queremos

Venga, vamos a ponernos un poco más serios. Veamos el pseudo-algoritmo:

**START**

Generate the initial population

Compute fitness

**REPEAT**
    
    Selection
    
    Crossover
    
    Mutation

    Compute fitness

**UNTIL** population has converged

**STOP**

Se entiende, no? Es exactamente lo mismo que los pasos que acabamos de hablar con el puzzle!!

<img src="https://image.ibb.co/kQu7Ed/ga_workflow.png" alt="ga_workflow" border="0" width="600">

Vale, y ahora llega lo interesante, cómo funcionan realmente?

Tenemos que entender varios conceptos:

* cómo se inicializa nuestra población
* cómo funciona el crossover
* cómo funciona la mutación
* cómo funciona la seleccion
* cómo podemos definir nuestra función de fitness

Preparados? Vamos allá!

Lo primero es entender que cuando tenemos un problema en el mundo real y queremos solucionarlo en un ordenador, necesitamos **codificarlo** para que el ordenador lo entienda.

Así, por ejemplo:

* En el mundo real, el cromosoma es la pieza del puzzle. En el ordenador, el cromosoma es un vector con 4 valores (uno para indicar el tamaño de cada punta, donde positivo significa punta, negativo significa hueco hacia dentro de la pieza)

Esto es a lo que se llama la codificación.

Una vez sabido esto, vamos a ver cómo funcionan los operadores. Antes de nada, debéis saber que xisten muchos tipos de crossover, mutación y selección, pero aquí vamos a ver solo los más sencillos por temas de tiempo. 

Si estáis interesados en conocerlos más profundamente, en internet hay muchísimo material disponible. Podéis empezar aquí: https://www.tutorialspoint.com/genetic_algorithms/index.htm

Vale, al turrón!

<img src="https://image.ibb.co/hEwVHd/ga_operators.png" alt="ga_operators" border="0">

**El crossover de un único punto (single-point crossover)**

Nuestro cromosoma es la pieza de puzzle, que tiene los 4 genes que veis en la imagen. Pues el crossover simple sencillamente escoge un punto aleatoriamente de los 4 genes, y combina las partes en nuevos cromosomas, como veis en la imagen.

Es importante que entendáis que el crossover **produce nuevos cromosomas**, puesto que tenemos los originales y los **recombinados**.

**La mutación uniforme**

La mutación uniforme consiste en que para cada cromosoma, lanzamos una moneda al aire. Si sale cara, modificamos un gen escogido aleatoriamente. ¿Qué valor le asignamos? Uno aleatorio dentro del rango que permite dicho gen.

**La selección**

Para la selección lo que se suele hacer es usar las fitness de los cromosomas (también llamados posibles **soluciones**). En este caso, vamos a ver la Stochastic Universal Sampling, que consiste en que construímos una gráfica de tipo tarta donde cada cromosoma ocupa un espacio que corresponde con su fitness. Después, establecemos N puntos fijos alrededor de la "tarta", donde N son el número de cromomsomas que queremos **seleccionar**. Después, se "gira la tarta", como si fuese la ruleta de la suerte, y los cromosomas a los que apuntan los puntos fijos son los seleccionados y pasan a la siguiente iteración.

Si os fijáis, **los cromosomas no están ordenados de mayor a menor fitness**!! 

Esto es importante, puesto que si no, las probabilidades de seleccionar un cromosoma con una fitness alta y otro con una fitness baja serían más altas que de seleccionar dos con la fitness alta, ya que al estar los puntos de selección uno enfrente del otro, sería muy complicado seleccionar dos cromosomas con fitness parecidas.

Este operador tiene varias formas de funcionar. Siguiendo con nuestro ejemplo de población de 10 cromosomas, las formas son:

* Seleccionamos $N=10$ cromosomas, es decir, sustituimos la anterior población por una completamente nueva
* Seleccionamos $N=n$ cromosomas, donde n<10. Es decir, sustituímos solo una parte de los crosomomas antiguos. El resto siguen jugando ;-D

Vale, pues si seleccionamos los 10 está claro, pero si seleccionamos $n$, cómo elegimos cuales quitamos?

Pues las dos formas más comunes son:

* Quitamos los cromosomas más antiguos
* Quitamos los cromosomas con peor fitness

Ya por último, hay veces que seleccionamos al mejor cromosoma (o a los $k$ mejors) para que pase si o si a la siguiente iteración, es decir, hay *elitismo*. Hay que ir con cuidado con esto, puesto que aunque a priori parezca que el elitismo es lo mejor y que solo deberíamos quedarnos con los mejores, si lo hiciésemos nos estaríamos cargando una de las mayores virtudes de los genéticos: **que son capaces de escapar a mínimos locales!!!**

Fijaos, aquí podéis ver en plena acción a un genético intentando decidir cual es la mejor configuración para un vehículo de 2 ruedas: http://rednuht.org/genetic_cars_2/

Curioso, eh?

Vale, pues ahora qué tal si implementamos un par de ejemplos nosotros?





In [15]:
# ejemplo simple de un GA donde tenemos que encontrar N números que sumen X
# https://lethain.com/genetic-algorithms-cool-name-damn-simple

from random import randint, random
from operator import add
from functools import reduce
import numpy as np

def individual(length, min, max):
    # creamos un individuo
    return [ randint(min,max) for x in range(length) ]

def population(count, length, min, max):   
    # creamos nuestra población
    # count: numero de invidiuos en la población
    # length: número de valores por individuo
    # min: minimo permitido para cada valor del individuo
    # max: maximo permitido para cada valor del individuo

    return [ individual(length, min, max) for x in range(count) ]

def fitness(individual, target):
    # calculamos la fitness de un individuo: más bajo es mejor
    
    sum = reduce(add, individual, 0)
    return abs(target-sum)

def grade(pop, target):
    # calculamos la media de la población entera
    summed = reduce(add, (fitness(x, target) for x in pop))
    return summed / (len(pop) * 1.0)
  
def find_best_solution(pop, target):
    # encuentra la mejor solución en la población actual y la imprime
    res = [fitness(x, target) for x in pop]
    res_min = np.min(res)
    res_min_idx = np.where(res == res_min)[0]
    for n in res_min_idx:
        print('Individual: ', n, 'Valores: ', *pop[n], ' Result: ', np.sum(pop[n]), 'Target; ', target)
    return res_min    

def evolve(pop, target, retain=0.2, random_select=0.05, mutate=0.01):
    graded = [ (fitness(x, target), x) for x in pop]
    graded = [ x[1] for x in sorted(graded)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]
    
    # añadimos individuos aleatoriamente para promover la diversidad genética
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)
    
    # mutamos algunos
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            individual[pos_to_mutate] = randint(i_min, i_max)
    
    # reproducimos (crossover) nuestros cromosomas (individuals, soluciones)
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = round(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)        
    parents.extend(children)
    return parents

# ejecutamos el GA
generations = 20
target = 108
p_count = 20
i_length = 5
i_min = 0
i_max = 100
error_accepted = 1
print('Inicializamos la población con 20 individuos.')
p = population(p_count, i_length, i_min, i_max)
print('Calculamos el fitness de esos 20 individuos.')
fitness_history = [grade(p, target),]
print('El mejor individuo de la población inicial es:')
find_best_solution(p, target)

for i in range(generations):
    p = evolve(p, target, retain=0.2, random_select=0.2, mutate=0.4)
    res = grade(p, target)
    fitness_history.append(res)
    
    res_min = find_best_solution(p, target)
    print('Generación: ', i, ' Fitness medio de todos los individuos de la población:', res)
    
    if res_min < error_accepted:
      break

Inicializamos la población con 20 individuos.
Calculamos el fitness de esos 20 individuos.
El mejor individuo de la población inicial es:
Individual:  11 Valores:  50 16 20 11 21  Result:  118 Target;  108
Individual:  8 Valores:  31 26 20 11 21  Result:  109 Target;  108
Generación:  0  Fitness medio de todos los individuos de la población: 75.85
Individual:  0 Valores:  31 26 20 11 21  Result:  109 Target;  108
Generación:  1  Fitness medio de todos los individuos de la población: 61.1
Individual:  0 Valores:  31 26 20 11 21  Result:  109 Target;  108
Generación:  2  Fitness medio de todos los individuos de la población: 51.8
Individual:  0 Valores:  31 26 20 11 21  Result:  109 Target;  108
Individual:  6 Valores:  7 50 20 11 21  Result:  109 Target;  108
Generación:  3  Fitness medio de todos los individuos de la población: 45.5
Individual:  0 Valores:  7 50 20 11 21  Result:  109 Target;  108
Individual:  1 Valores:  31 26 20 11 21  Result:  109 Target;  108
Individual:  15 Valore

In [16]:
# Ejemplo un poco más complejo de un GA que tiene que encontrar la cadena 'Hello Word!'
# http://www.obitko.com/tutorials/genetic-algorithms/ga-basic-description.php

import random

class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics = genetics
        pass

    def run(self):
        population = self.genetics.initial()
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops): break
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

class GeneticFunctions(object):
    def probability_crossover(self):
        # returns rate of occur crossover(0.0-1.0)
        return 1.0

    def probability_mutation(self):
        # returns rate of occur mutation(0.0-1.0)
        return 0.0

    def initial(self):
        # returns list of initial population        
        return []

    def fitness(self, chromosome):
        # returns domain fitness value of chromosome
        return len(chromosome)

    def check_stop(self, fits_populations):
        # stop run if returns True
        # - fits_populations: list of (fitness_value, chromosome)
        return False

    def parents(self, fits_populations):
        r"""generator of selected parents
        """
        gen = iter(sorted(fits_populations))
        while True:
            f1, ch1 = next(gen)
            f2, ch2 = next(gen)
            yield (ch1, ch2)
            pass
        return

    def crossover(self, parents):
        r"""breed children
        """
        return parents

    def mutation(self, chromosome):
        r"""mutate chromosome
        """
        return chromosome
    pass

if __name__ == "__main__":
    """
    example: Mapped guess prepared Text
    """
    class GuessText(GeneticFunctions):
        def __init__(self, target_text,
                     limit=200, size=400,
                     prob_crossover=0.9, prob_mutation=0.2):
            self.target = self.text2chromo(target_text)
            self.counter = 0

            self.limit = limit
            self.size = size
            self.prob_crossover = prob_crossover
            self.prob_mutation = prob_mutation
            pass

        # GeneticFunctions interface impls
        def probability_crossover(self):
            return self.prob_crossover

        def probability_mutation(self):
            return self.prob_mutation

        def initial(self):
            return [self.random_chromo() for j in range(self.size)]

        def fitness(self, chromo):
            # larger is better, matched == 0
            return -sum(abs(c - t) for c, t in zip(chromo, self.target))

        def check_stop(self, fits_populations):
            self.counter += 1
            if self.counter % 10 == 0:
                best_match = list(sorted(fits_populations))[-1][1]
                fits = [f for f, ch in fits_populations]
                best = max(fits)
                worst = min(fits)
                ave = sum(fits) / len(fits)
                print(
                    "[G %3d] score=(%4d, %4d, %4d): %r" %
                    (self.counter, best, ave, worst,
                     self.chromo2text(best_match)))
                pass
            return self.counter >= self.limit

        def parents(self, fits_populations):
            while True:
                father = self.tournament(fits_populations)
                mother = self.tournament(fits_populations)
                yield (father, mother)
                pass
            pass

        def crossover(self, parents):
            father, mother = parents
            index1 = random.randint(1, len(self.target) - 2)
            index2 = random.randint(1, len(self.target) - 2)
            if index1 > index2: index1, index2 = index2, index1
            child1 = father[:index1] + mother[index1:index2] + father[index2:]
            child2 = mother[:index1] + father[index1:index2] + mother[index2:]
            return (child1, child2)

        def mutation(self, chromosome):
            index = random.randint(0, len(self.target) - 1)
            vary = random.randint(-5, 5)
            mutated = list(chromosome)
            mutated[index] += vary
            return mutated

        # internals
        def tournament(self, fits_populations):
            alicef, alice = self.select_random(fits_populations)
            bobf, bob = self.select_random(fits_populations)
            return alice if alicef > bobf else bob

        def select_random(self, fits_populations):
            return fits_populations[random.randint(0, len(fits_populations)-1)]

        def text2chromo(self, text):
            return [ord(ch) for ch in text]
        def chromo2text(self, chromo):
            return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

        def random_chromo(self):
            return [random.randint(1, 255) for i in range(len(self.target))]
        pass

    GeneticAlgorithm(GuessText("Hello World!")).run()
    pass

[G  10] score=(-169, -338, -526): "_gSbl\x1e4{Wjz'"
[G  20] score=( -66, -125, -195): '_ehhl Ymyju#'
[G  30] score=( -55,  -64,  -79): '[gkpp\x1dWpqgu '
[G  40] score=( -34,  -44,  -57): 'Rglpp Wprjp#'
[G  50] score=( -20,  -29,  -36): 'Pekln Wprlk#'
[G  60] score=( -10,  -18,  -25): 'Kellp!Wormf#'
[G  70] score=(  -4,   -8,  -15): 'Hello Worlf#'
[G  80] score=(   0,   -3,  -10): 'Hello World!'
[G  90] score=(   0,    0,   -8): 'Hello World!'
[G 100] score=(   0,    0,   -9): 'Hello World!'
[G 110] score=(   0,    0,   -6): 'Hello World!'
[G 120] score=(   0,    0,   -7): 'Hello World!'
[G 130] score=(   0,    0,   -5): 'Hello World!'
[G 140] score=(   0,    0,   -7): 'Hello World!'
[G 150] score=(   0,    0,   -5): 'Hello World!'
[G 160] score=(   0,    0,   -9): 'Hello World!'
[G 170] score=(   0,    0,   -6): 'Hello World!'
[G 180] score=(   0,    0,   -7): 'Hello World!'
[G 190] score=(   0,    0,   -7): 'Hello World!'
[G 200] score=(   0,    0,   -7): 'Hello World!'


Fijaos como es capaz de llegar a la cadena "Hello World!" tras unas pocas generaciones!

Y ahora, vamos a verlo aplicado a una red neuronal. Para ello, vamos a hacer uso de una implementación disponible en Github: https://github.com/jliphard/DeepEvolve

Para ello, vamos a clonar un repositorio con git que tiene implementado un GA para evolucionar los hiperparámetros y arquitectura de una red neuronal:

In [16]:
!rm -rf DeepEvolve
!git clone https://github.com/jliphard/DeepEvolve.git

Cloning into 'DeepEvolve'...
remote: Counting objects: 152, done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 152 (delta 4), reused 2 (delta 0), pack-reused 140
Receiving objects: 100% (152/152), 1.52 MiB | 7.70 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [17]:
!ls

datalab  DeepEvolve


In [18]:
!pip install tqdm

Quiero que veáis los parámetros entre los que va a buscar hasta encontrar la mejor combinación posible:

In [0]:
if dataset == 'mnist_cnn':
        generations = 8 # Number of times to evolve the population.
        all_possible_genes = {
            'nb_neurons': [16, 32, 64, 128],
            'nb_layers':  [1, 2, 3, 4 ,5],
            'activation': ['relu', 'elu', 'tanh', 'sigmoid', 'hard_sigmoid','softplus','linear'],
            'optimizer':  ['rmsprop', 'adam', 'sgd', 'adagrad','adadelta', 'adamax', 'nadam']
        }

Y ahora ejecutamos nuestro algoritmo genético:

In [19]:
!python DeepEvolve/main.py

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
***Dataset: cifar10_cnn
***Evolving for 8 generations with population size = 30***
06/16/2018 08:06:22 PM - INFO - ***generate(generations, population, all_possible_genes, dataset)***
06/16/2018 08:06:22 PM - INFO - ***Now in generation 1 of 8***
06/16/2018 08:06:22 PM - INFO - --------------------------------------------------------------------------------
06/16/2018 08:06:22 PM - INFO - {'nb_layers': 3, 'activation': 'elu', 'optimizer': 'adagrad', 'nb_neurons': [128, 64, 32, 16, 16, 16]}
06/16/2018 08:06:22 PM - INFO - Acc: 0.00%
06/16/2018 08:06:22 PM - INFO - UniID: 1
06/16/2018 08:06:22 PM - INFO - Mom and Dad: 0 0
06/16/2018 08:06:22 PM - INFO - Gen: 1
06/

50000/50000 [==============================] - 15s 295us/step - loss: 2.0599 - acc: 0.2699 - val_loss: 1.6352 - val_acc: 0.4442
Epoch 2/4
50000/50000 [==============================] - 13s 255us/step - loss: 1.7225 - acc: 0.3740 - val_loss: 1.4797 - val_acc: 0.4779
Epoch 3/4
28288/50000 [===============>..............] - ETA: 5s - loss: 1.6513 - acc: 0.4029

50000/50000 [==============================] - 13s 256us/step - loss: 1.6350 - acc: 0.4087 - val_loss: 1.4008 - val_acc: 0.5187
Epoch 4/4
50000/50000 [==============================] - 13s 256us/step - loss: 1.5700 - acc: 0.4334 - val_loss: 1.3358 - val_acc: 0.5397
Test loss: 1.335766393661499
Test accuracy: 0.5397
  3%|█▍                                          | 1/30 [00:56<27:12, 56.30s/it]06/16/2018 08:07:18 PM - INFO - Getting Keras datasets
06/16/2018 08:07:19 PM - INFO - Compling Keras model
06/16/2018 08:07:19 PM - INFO - Architecture:[64, 32, 64, 128, 64, 32],relu,sgd,2
Train on 50000 samples, validate on 10000 samples
Epoch 1/4
2018-06-16 20:07:20.053220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-16 20:07:20.053338: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-16 20:07:20.053379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0

50000/50000 [==============================] - 9s 175us/step - loss: 2.2881 - acc: 0.1220 - val_loss: 2.2483 - val_acc: 0.1697
Epoch 2/4
50000/50000 [==============================] - 8s 168us/step - loss: 2.2156 - acc: 0.1758 - val_loss: 2.1338 - val_acc: 0.2371
Epoch 3/4
50000/50000 [==============================] - 9s 172us/step - loss: 2.1422 - acc: 0.2084 - val_loss: 2.0599 - val_acc: 0.2879
Epoch 4/4
10880/50000 [=====>........................] - ETA: 6s - loss: 2.1092 - acc: 0.2142

50000/50000 [==============================] - 9s 173us/step - loss: 2.0901 - acc: 0.2277 - val_loss: 2.0124 - val_acc: 0.3151
Test loss: 2.0123611381530764
Test accuracy: 0.3151
  7%|██▉                                         | 2/30 [01:33<23:35, 50.54s/it]06/16/2018 08:07:55 PM - INFO - Getting Keras datasets
06/16/2018 08:07:56 PM - INFO - Compling Keras model
06/16/2018 08:07:56 PM - INFO - Architecture:[32, 128, 16, 128, 128, 64],hard_sigmoid,rmsprop,5
Train on 50000 samples, validate on 10000 samples
Epoch 1/4
2018-06-16 20:07:57.612455: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-16 20:07:57.612552: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-16 20:07:57.612592: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-16 20:07:57.612660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-16 20:07:57.612879: I 

...

50000/50000 [==============================] - 19s 377us/step - loss: 1.2728 - acc: 0.5574 - val_loss: 1.0631 - val_acc: 0.6297
Epoch 4/4
50000/50000 [==============================] - 19s 381us/step - loss: 1.2172 - acc: 0.5762 - val_loss: 1.0270 - val_acc: 0.6446
Test loss: 1.0270381179809571
Test accuracy: 0.6446
 94%|████████████████████████████████████████▍  | 32/34 [28:11<02:23, 71.55s/it]06/16/2018 11:37:24 PM - INFO - Getting Keras datasets
06/16/2018 11:37:25 PM - INFO - Compling Keras model
06/16/2018 11:37:25 PM - INFO - Architecture:[16, 128, 128, 128, 32, 128],relu,adam,3
Train on 50000 samples, validate on 10000 samples
Epoch 1/4
2018-06-16 23:37:26.558769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-16 23:37:26.558962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-16 23:37:26.559047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]  

50000/50000 [==============================] - 15s 292us/step - loss: 1.7745 - acc: 0.3413 - val_loss: 1.4128 - val_acc: 0.5033
Epoch 2/4
50000/50000 [==============================] - 15s 291us/step - loss: 1.4313 - acc: 0.4821 - val_loss: 1.2046 - val_acc: 0.5726
Epoch 3/4


50000/50000 [==============================] - 14s 282us/step - loss: 1.2933 - acc: 0.5356 - val_loss: 1.1396 - val_acc: 0.6015
Epoch 4/4
50000/50000 [==============================] - 15s 292us/step - loss: 1.2017 - acc: 0.5745 - val_loss: 1.0589 - val_acc: 0.6280
Test loss: 1.0588940828323363
Test accuracy: 0.628
 97%|█████████████████████████████████████████▋ | 33/34 [29:12<01:08, 68.46s/it]06/16/2018 11:38:26 PM - INFO - Getting Keras datasets
06/16/2018 11:38:26 PM - INFO - Compling Keras model
06/16/2018 11:38:26 PM - INFO - Architecture:[128, 128, 128, 16, 32, 128],softplus,adam,3
Train on 50000 samples, validate on 10000 samples
Epoch 1/4
2018-06-16 23:38:28.175940: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-16 23:38:28.176126: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-16 23:38:28.176180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929

50000/50000 [==============================] - 20s 399us/step - loss: 2.3211 - acc: 0.0994 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 2/4
50000/50000 [==============================] - 19s 380us/step - loss: 2.3027 - acc: 0.0995 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 3/4


50000/50000 [==============================] - 19s 385us/step - loss: 2.3027 - acc: 0.0977 - val_loss: 2.3026 - val_acc: 0.1000
Test loss: 2.302590662765503
Test accuracy: 0.1
100%|███████████████████████████████████████████| 34/34 [30:14<00:00, 66.62s/it]
06/16/2018 11:39:28 PM - INFO - Generation average: 57.67%
06/16/2018 11:39:28 PM - INFO - --------------------------------------------------------------------------------
06/16/2018 11:39:28 PM - INFO - --------------------------------------------------------------------------------
06/16/2018 11:39:28 PM - INFO - {'activation': 'relu', 'nb_layers': 3, 'optimizer': 'nadam', 'nb_neurons': [32, 128, 128, 32, 64, 128]}
06/16/2018 11:39:28 PM - INFO - Acc: 69.25%
06/16/2018 11:39:28 PM - INFO - UniID: 186
06/16/2018 11:39:28 PM - INFO - Mom and Dad: 172 174
06/16/2018 11:39:28 PM - INFO - Gen: 8
06/16/2018 11:39:28 PM - INFO - Hash: 3e565cd3e8109a7ba76398215343f6e7
06/16/2018 11:39:28 PM - INFO - {'activation': 'relu', 'nb_layers': 3, '

Y os recomiendo que le echéis un ojo a estos dos enlaces en los que evolucionan una red neuronal con un GA. Aquí no nos da tiempo, pero son muy interesantes:

### Ejempo GA para evolucionar NN: 
* https://blog.coast.ai/lets-evolve-a-neural-network-with-a-genetic-algorithm-code-included-8809bece164
* https://github.com/harvitronix/neural-network-genetic-algorithm